# Age, gender, race prediction with CNN

## Importing modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from keras.models import Sequential,load_model,Model
from keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,Flatten,Input
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input,Activation
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint

## Preprocessing

In [ ]:
# Load dataset
dir = "./utk/UTKFace/"
imgs = []
age = []
gender = [] 
race = []

for img in os.listdir(dir):
  ages = img.split("_")[0]
  genders = img.split("_")[1]
  races = img.split("_")[2]
  img = cv2.imread(str(dir)+"/"+str(img))
  img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
  img=cv2.resize(img,(100,100))
  imgs.append(np.array(img))
  age.append(np.array(ages))
  gender.append(np.array(genders))
  race.append(np.array(races))

imgs, age, gender, race = np.array(imgs), np.array(age,np.uint64), np.array(gender,np.uint64), np.array(race,np.uint64)

x_train_age,x_test_age,y_train_age,y_test_age = train_test_split(imgs,age,random_state=100, test_size=0.2)
x_train_gender,x_test_gender,y_train_gender,y_test_gender = train_test_split(imgs,gender,random_state=100, test_size=0.2)
x_train_race,x_test_race,y_train_race,y_test_race = train_test_split(imgs,race,random_state=100, test_size=0.2)

In [ ]:
for x in range(len(y_train_age)):
    if y_train_age[x] < 18:
        y_train_age[x] = 0
    elif y_train_age[x] < 30:
        y_train_age[x] = 1
    elif y_train_age[x] < 40:
        y_train_age[x] = 2
    elif y_train_age[x] < 50:
        y_train_age[x] = 3
    elif y_train_age[x] < 60:
        y_train_age[x] = 4
    else:
        y_train_age[x] = 5 

for x in range(len(y_test_age)):
    if y_test_age[x] < 18:
        y_test_age[x] = 0
    elif y_test_age[x] < 30:
        y_test_age[x] = 1
    elif y_test_age[x] < 40:
        y_test_age[x] = 2
    elif y_test_age[x] < 50:
        y_test_age[x] = 3
    elif y_test_age[x] < 60:
        y_test_age[x] = 4
    else:
        y_test_age[x] = 5 

## CNN Models
For every label there is a different model trained

### Gender

In [ ]:
# CNN gender Architecture
x = Input(shape = (100,100,1))
x = Conv2D(32,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(x)
x = Dropout(0.1)(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2)) (x)
x = Conv2D(64,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(x)
x = Dropout(0.1)(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2)) (x)
x = Conv2D(128,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(x)
x = Dropout(0.1)(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2)) (x)
x = Conv2D(256,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(x)
x = Dropout(0.1)(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2)) (x)
x = Flatten()(x_train_gender)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)
output_gender = Dense(2,activation="sigmoid")(x)


model_gender = Model(inputs=input,outputs=output_gender)
model_gender.compile(optimizer="adam",loss=["sparse_categorical_crossentropy"],metrics=['accuracy'])
model_gender.summary()


model_path='./gender_model.h5'
checkpointer = ModelCheckpoint(model_path, monitor='loss',verbose=1,save_best_only=True,
                               save_weights_only=False, mode='auto',save_freq='epoch')
callback_list=[checkpointer]

save_gender = model_gender.fit(x_train_gender,y_train_gender,validation_data=(x_test_gender,y_test_gender),epochs=30,callbacks=[callback_list])

Loss and accuracy graphs for evaluation

In [ ]:

train_loss = save_gender.history['loss']
test_loss = save_gender.history['val_loss']
train_accuracy = save_gender.history['accuracy']
test_accuracy = save_gender.history['val_accuracy']


fig, ax = plt.subplots(ncols=2, figsize=(15,7))
ax = ax.ravel()
ax[0].plot(train_loss, label='Train Loss', color='b')
ax[0].plot(test_loss, label='Test Loss', color = 'r')
ax[0].set_xlabel('Epochs', fontsize=14)
ax[0].set_ylabel('Categorical Crossentropy', fontsize=14)
ax[0].legend(fontsize=14)
ax[0].tick_params(axis='both', labelsize=12)
ax[1].plot(train_accuracy, label='Train Accuracy', color='b')
ax[1].plot(test_accuracy, label='Test Accuracy', color='r')
ax[1].set_xlabel('Epochs', fontsize=14)
ax[1].set_ylabel('Accuracy', fontsize=14)
ax[1].legend(fontsize=14)
ax[1].tick_params(axis='both', labelsize=12)


### Age

In [ ]:
# CNN age Architecture
x = Input(shape = (100,100,1))
x = Conv2D(32,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(x)
x = Dropout(0.1)(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2)) (x)
x = Conv2D(64,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(x)
x = Dropout(0.1)(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2)) (x)
x = Conv2D(128,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(x)
x = Dropout(0.1)(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2)) (x)
x = Conv2D(256,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(x)
x = Dropout(0.1)(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2)) (x)
x = Flatten()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)
output_age = Dense(6,activation="softmax")(x)

# Model compile
model_age = Model(inputs=input,outputs=output_age)
model_age.compile(optimizer="adamax",loss=["sparse_categorical_crossentropy"],metrics=['accuracy'])
#model_age.summary()

# Model Checkpoint
model_path='./age_model.h5'
checkpointer = ModelCheckpoint(model_path, monitor='loss',verbose=1,save_best_only=True,
                               save_weights_only=False, mode='auto',save_freq='epoch')
callback_list=[checkpointer]

save_age = model_age.fit(x_train_age,y_train_age,validation_data=(x_test_age,y_test_age),epochs=30,callbacks=[callback_list])

Loss and accuracy graphs for evaluation

In [ ]:

train_loss = save_age.history['loss']
test_loss = save_age.history['val_loss']
train_accuracy = save_age.history['accuracy']
test_accuracy = save_age.history['val_accuracy']

fig, ax = plt.subplots(ncols=2, figsize=(15,7))
ax = ax.ravel()
ax[0].plot(train_loss, label='Train Loss', color='b')
ax[0].plot(test_loss, label='Test Loss', color = 'r')
ax[0].set_xlabel('Epochs', fontsize=14)
ax[0].set_ylabel('Categorical Crossentropy', fontsize=14)
ax[0].legend(fontsize=14)
ax[0].tick_params(axis='both', labelsize=12)
ax[1].plot(train_accuracy, label='Train Accuracy', color='b')
ax[1].plot(test_accuracy, label='Test Accuracy', color='r')
ax[1].set_xlabel('Epochs', fontsize=14)
ax[1].set_ylabel('Accuracy', fontsize=14)
ax[1].legend(fontsize=14)
ax[1].tick_params(axis='both', labelsize=12)


### Race

In [ ]:
x = Input(shape = (100,100,1))
x = Conv2D(32,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(x)
x = Dropout(0.1)(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2)) (x)
x = Conv2D(64,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(x)
x = Dropout(0.1)(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2)) (x)
x = Conv2D(128,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(x)
x = Dropout(0.1)(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2)) (x)
x = Conv2D(256,(3, 3), padding = 'same', strides=(1, 1), kernel_regularizer=l2(0.001))(x)
x = Dropout(0.1)(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size = (2,2)) (x)
x = Flatten()(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.2)(x)
output_race = Dense(5,activation="softmax")(x)

# Model compile
model_race = Model(inputs=input,outputs=output_race)
model_race.compile(optimizer="adam",loss=["sparse_categorical_crossentropy"],metrics=['accuracy'])
#model_race.summary()

# Model Checkpoint
model_path='./race_model.h5'
checkpointer = ModelCheckpoint(model_path, monitor='loss',verbose=1,save_best_only=True,
                               save_weights_only=False, mode='auto',save_freq='epoch')
callback_list=[checkpointer]

save_race = model_race.fit(x_train_race,y_train_race,validation_data=(x_test_race,y_test_race),epochs=30,callbacks=[callback_list])

Loss and accuracy graphs for evaluation

In [ ]:

train_loss = save_race.history['loss']
test_loss = save_race.history['val_loss']
train_accuracy = save_race.history['accuracy']
test_accuracy = save_race.history['val_accuracy']

fig, ax = plt.subplots(ncols=2, figsize=(15,7))
ax = ax.ravel()
ax[0].plot(train_loss, label='Train Loss', color='b')
ax[0].plot(test_loss, label='Test Loss', color = 'r')
ax[0].set_xlabel('Epochs', fontsize=14)
ax[0].set_ylabel('Categorical Crossentropy', fontsize=14)
ax[0].legend(fontsize=14)
ax[0].tick_params(axis='both', labelsize=12)
ax[1].plot(train_accuracy, label='Train Accuracy', color='b')
ax[1].plot(test_accuracy, label='Test Accuracy', color='r')
ax[1].set_xlabel('Epochs', fontsize=14)
ax[1].set_ylabel('Accuracy', fontsize=14)
ax[1].legend(fontsize=14)
ax[1].tick_params(axis='both', labelsize=12)
